In [6]:
from siphon.simplewebservice.wyoming import WyomingUpperAir
from datetime import datetime

start_time = datetime(2023, 3, 1, 0)
end_time = datetime(2023, 6, 30, 0)

station = "43371" # Trivandrum, Thiruvananthapuram, 'VOTX'

df = WyomingUpperAir.request_data(start_time, station) # returns Pandas dataframe
# attributes like units are lost???

In [7]:
df

,pressure,height,temperature,dewpoint,direction,speed,u_wind,v_wind,station,station_number,time,latitude,longitude,elevation,pw
0,1005.0,64,26.2,22.1,0.0,0.0,-0.000000,-0.000000e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
1,1000.0,109,25.8,22.8,30.0,2.0,-1.000000,-1.732051e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
2,925.0,793,22.0,17.0,80.0,4.0,-3.939231,-6.945927e-01,,43371,2023-03-01,8.48,76.95,64.0,38.48
3,900.0,1030,20.2,16.4,75.0,10.0,-9.659258,-2.588190e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
4,850.0,1523,16.4,15.3,75.0,15.0,-14.488887,-3.882286e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
5,848.0,1543,16.2,15.2,75.0,15.0,-14.488887,-3.882286e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
6,841.0,1614,15.6,15.0,80.0,15.0,-14.772116,-2.604723e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
7,804.0,1992,13.8,12.7,105.0,17.0,-16.420739,4.399924e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48
8,753.0,2543,11.1,9.3,90.0,21.0,-21.000000,-1.285879e-15,,43371,2023-03-01,8.48,76.95,64.0,38.48
9,738.0,2712,10.3,8.2,110.0,23.0,-21.612930,7.866463e+00,,43371,2023-03-01,8.48,76.95,64.0,38.48


In [62]:
for column in df:
    print( df[column].name )
    print( df[column].values )

# df[column].shape[0]
df[column]
# df[df.columns[0]].shape[0]

pressure
[1005. 1000.  925.  900.  850.  848.  841.  804.  753.  738.  708.]
height
[  64  109  793 1030 1523 1543 1614 1992 2543 2712 3060]
temperature
[26.2 25.8 22.  20.2 16.4 16.2 15.6 13.8 11.1 10.3  8.6]
dewpoint
[22.1 22.8 17.  16.4 15.3 15.2 15.  12.7  9.3  8.2  6.1]
direction
[  0.  30.  80.  75.  75.  75.  80. 105.  90. 110.  nan]
speed
[ 0.  2.  4. 10. 15. 15. 15. 17. 21. 23. nan]
u_wind
[ -0.          -1.          -3.93923101  -9.65925826 -14.48888739
 -14.48888739 -14.7721163  -16.42073905 -21.         -21.61293028
          nan]
v_wind
[-0.00000000e+00 -1.73205081e+00 -6.94592711e-01 -2.58819045e+00
 -3.88228568e+00 -3.88228568e+00 -2.60472267e+00  4.39992377e+00
 -1.28587914e-15  7.86646330e+00             nan]
station
['' '' '' '' '' '' '' '' '' '' '']
station_number
[43371 43371 43371 43371 43371 43371 43371 43371 43371 43371 43371]
time
['2023-03-01T00:00:00.000000000' '2023-03-01T00:00:00.000000000'
 '2023-03-01T00:00:00.000000000' '2023-03-01T00:00:00.000000000'
 '2

0     38.48
1     38.48
2     38.48
3     38.48
4     38.48
5     38.48
6     38.48
7     38.48
8     38.48
9     38.48
10    38.48
Name: pw, dtype: float64

In [17]:
# bad way... & broken

#from siphon.cdmr import Dataset
from siphon.ncss import Dataset
from io import StringIO

# Convert the DataFrame to CSV format in memory
csv_buffer = StringIO()
df.to_csv(csv_buffer, sep=',', index=False)
csv_buffer.getvalue().splitlines()[0].encode()
csv_buffer.getvalue().splitlines()[1:]

# Open a Dataset object and write the CSV data to it
nc = Dataset('sounding.nc', 'w', format='NETCDF4')
nc.createDimension('index', None)
nc.createVariable('header', 'S1', ('header',))
# nc.variables['header'][0] = csv_buffer.getvalue().splitlines()[0].encode()
# nc.variables['header'][0] = 
nc.createVariable('data', 'f8', ('index', 'header'))

for i, column in enumerate(df):
    # print( df[column].name )
    nc.variables['header'][i] = df[column].name
    # print( df[column].values )
    
for i, line in enumerate(csv_buffer.getvalue().splitlines()[1:]):
    #nc.variables['data'][i, :] = line.split(',') # can't autoconvert string to float
    line.split(',')
nc.close()


In [ ]:
from netCDF4 import Dataset

# Open a new NetCDF file for writing
nc_file = Dataset('sounding.nc', mode='w')


# Define dimensions for pressure, temperature, etc. variables
level_dim = nc_file.createDimension('level', df[df.columns[0]].shape[0])

# Create variables for time, level, latitude, longitude, pressure, and temperature
level_var = nc_file.createVariable('level', 'f4', ('level',))
level_var.units = 'hPa'
lat_var = nc_file.createVariable('lat', 'f4', ('lat',))
lat_var.units = 'degrees_north'
lon_var = nc_file.createVariable('lon', 'f4', ('lon',))
lon_var.units = 'degrees_east'
for column in df:
pressure_var = nc_file.createVariable(df, 'f4', ('time', 'level', 'lat', 'lon',))
pressure_var.units = 'hPa'
temperature_var = nc_file.createVariable('temperature', 'f4', ('time', 'level', 'lat', 'lon',))
temperature_var.units = 'K'

# Write the data to the NetCDF file
time_var[:] = data.variables['time'][:]
level_var[:] = data.variables['Pressure_reduced_to_MSL'][:].squeeze()
lat_var[:] = data.variables['lat'][:]
lon_var[:] = data.variables['lon'][:]
pressure_var[:] = data.variables['Pressure_reduced_to_MSL'][:]
temperature_var[:] = data.variables['Temperature_isobaric'][:]

# Close the NetCDF file
nc_file.close()